In [3]:
import numpy as np

spielfeldhälfte = 1 #1 oder -1. 1 ist bei team A
länge_des_ballwechsels = 0
#risiko_

class Team():
    def __init__(self, name, stärke, risiko = 0.5):
        self.name = name
        self.stärke = stärke
        self.punkte = 0
        self.risiko = risiko

    def onepunkt(self):
        self.punkte += 1

teams = dict()
teams["A"] = Team("TeamA", 0.8)
teams["B"] = Team("TeamB", 0.6)
risiko_list = [0.2, 0.4, 0.6, 0.8, 1]

for spiel in range(0,1):
    ballposession = "A"
    for risiko in risiko_list:
        print("Risiko: ",risiko)
        teams["A"].risiko = risiko
        teams["A"].punkte = 0
        teams["B"].punkte = 0
        while teams["A"].punkte<21 and teams["B"].punkte<21:
            #print("Ballwechsel Nr.",ballwechsel)
            länge_des_ballwechsels = 0
            while True:
                #ergebnis = np.random.randint(0, 100)
                ergebnis = np.random.normal(teams[ballposession].stärke, teams["A"].risiko)
                länge_des_ballwechsels += 1
                print("Länge des Ballwechsels:"+str(länge_des_ballwechsels))
                if ergebnis >= 0.5:      #Erfolgreicher Ball
                    spielfeldhälfte *= -1
                    if (spielfeldhälfte == 1):
                        ballposession = "A"
                    else:
                        ballposession = "B"
                else:
                    break
            if (spielfeldhälfte==1):
                teams["A"].onepunkt()
            else:
                teams["B"].onepunkt()
            print(teams["A"].name,str(teams["A"].punkte),teams["B"].name,str(teams["B"].punkte))
        print("Ende des Spiels.  Team A: ",teams["A"].punkte,"Team B:",teams["B"].punkte)


Risiko:  0.2
Länge des Ballwechsels:1
Länge des Ballwechsels:2
Länge des Ballwechsels:3
Länge des Ballwechsels:4
Länge des Ballwechsels:5
Länge des Ballwechsels:6
TeamA 0 TeamB 1
Länge des Ballwechsels:1
Länge des Ballwechsels:2
Länge des Ballwechsels:3
Länge des Ballwechsels:4
Länge des Ballwechsels:5
TeamA 0 TeamB 2
Länge des Ballwechsels:1
TeamA 0 TeamB 3
Länge des Ballwechsels:1
TeamA 0 TeamB 4
Länge des Ballwechsels:1
TeamA 0 TeamB 5
Länge des Ballwechsels:1
Länge des Ballwechsels:2
Länge des Ballwechsels:3
TeamA 0 TeamB 6
Länge des Ballwechsels:1
TeamA 0 TeamB 7
Länge des Ballwechsels:1
Länge des Ballwechsels:2
Länge des Ballwechsels:3
Länge des Ballwechsels:4
Länge des Ballwechsels:5
Länge des Ballwechsels:6
Länge des Ballwechsels:7
TeamA 0 TeamB 8
Länge des Ballwechsels:1
Länge des Ballwechsels:2
Länge des Ballwechsels:3
Länge des Ballwechsels:4
Länge des Ballwechsels:5
Länge des Ballwechsels:6
Länge des Ballwechsels:7
Länge des Ballwechsels:8
Länge des Ballwechsels:9
TeamA 0 T